In [1]:
import numpy as np

movies = open('hw8_movies.txt').read().splitlines()
student_ids = open('hw8_ids.txt').read().splitlines()
ratings_string = np.loadtxt('hw8_ratings.txt', dtype='str')

ratings = np.zeros(ratings_string.shape)

for i in range(ratings.shape[0]):
    for j in range(ratings.shape[1]):
        if ratings_string[i][j] == '?':
            ratings[i][j] = -1.0
        else:
            ratings[i][j] = float(ratings_string[i][j])

ratings = ratings.T
liked = ratings == 1
disliked = ratings == 0
unwatched = ratings == -1

# Part a.
sanity_check = np.sum(liked, axis = 1) / np.sum(liked + disliked, axis = 1)

popularity = []
for popular, movie in zip(sanity_check, movies):
    popularity.append((np.round(popular, 3), movie))

popularity.sort()

result = ", ".join([item[1] for item in popularity])
print(result)

Chappaquidick, I_Feel_Pretty, Fast_&_Furious:_Hobbs_&_Shaw, Magic_Mike, Bridemaids, The_Last_Airbender, World_War_Z, Fifty_Shades_of_Grey, Star_Wars:_The_Force_Awakens, The_Hateful_Eight, Terminator:_Dark_Fate, Drive, Pitch_Perfect, Phantom_Thread, Pokemon_Detective_Pikachu, Prometheus, Fast_Five, Good_Boys, The_Farewell, American_Hustle, The_Girl_with_the_Dragon_Tattoo, Avengers:_Age_of_Ultron, The_Shape_of_Water, The_Revenant, Thor, The_Help, Man_of_Steel, The_Perks_of_Being_a_Wallflower, Hidden_Figures, Frozen, Jurassic_World, Three_Billboards_Outside_Ebbing, Mad_Max:_Fury_Road, Captain_America:_The_First_Avenger, Dunkirk, Manchester_by_the_Sea, Ex_Machina, Rocketman, Ready_Player_One, Once_Upon_a_Time_in_Hollywood, 21_Jump_Street, Black_Swan, Iron_Man_2, Hustlers, Darkest_Hour, The_Hunger_Games, X-Men:_First_Class, Gone_Girl, Us, Avengers:_Endgame, La_La_Land, Avengers:_Infinity_War, Spiderman:_Far_From_Home, Midnight_in_Paris, Now_You_See_Me, 12_Years_a_Slave, Django_Unchained, Ro

In [11]:
# Part e.

probR = np.loadtxt('hw8_probR_init.txt')
probZ = np.loadtxt('hw8_probZ_init.txt')
probZ = probZ.reshape(-1,1)

for itr in range(257):
    liked_z = np.zeros((4, 76, 258))
    disliked_z = np.zeros((4, 76, 258))

    PRjZ_first = np.zeros((76,4))
    PRjZ_second = np.zeros((76,4))

    for i in range(4):
        liked_z[i] = np.multiply(probR[:,i].reshape(-1,1), liked)
        disliked_z[i] = np.multiply(1 - probR[:,i].reshape(-1,1), disliked)

    liked_zz = np.prod(liked_z, axis=1, where=(liked_z > 0))
    disliked_zz = np.prod(disliked_z, axis=1, where=(disliked_z > 0))

    numerator = np.multiply(np.multiply(liked_zz, disliked_zz), probZ)
    denominator = np.sum(numerator, axis=0).reshape(1,-1)
    pit = np.divide(numerator, denominator)

    if itr in [0,1,2,4,8,16,32,64,128,256]:
        print(itr, np.around(np.sum(np.log(denominator))/258, decimals=2))

    for i in range(4):
        PRjZ_first[:, i] = np.sum(np.multiply(liked, pit[i].reshape(1, -1)), axis=1)
        temp_unwatched = np.sum(np.multiply(unwatched, pit[i].reshape(1, -1)), axis=1)
        PRjZ_second[:, i] = np.multiply(temp_unwatched.reshape(-1, 1), probR[:, i].reshape(-1, 1)).flatten()

    probR_new = np.divide(PRjZ_first + PRjZ_second, np.sum(pit, axis=1).reshape(1,-1))
    probZ_new = np.sum(pit, axis=1).reshape(-1,1)/258

    probR = np.copy(probR_new)
    probZ = np.copy(probZ_new)

0 -27.62
1 -18.48
2 -16.79
4 -15.55
8 -14.98
16 -14.68
32 -14.56
64 -14.53
128 -14.53
256 -14.52


In [12]:
# Part f.

scores = np.matmul(probR, pit[:, 229].reshape(-1,1))

recommendations = []
for index, (movie, score) in enumerate(zip(movies, scores[:,0])):
    if ratings[index][229] == -1:
        recommendations.append((np.round(score, 2), movie))

recommendations.sort(reverse = True)
print(recommendations)

[(0.93, 'The_Theory_of_Everything'), (0.93, 'The_Social_Network'), (0.92, 'The_Dark_Knight_Rises'), (0.9, 'The_Martian'), (0.9, 'Hustlers'), (0.87, 'Darkest_Hour'), (0.86, 'Django_Unchained'), (0.85, 'Les_Miserables'), (0.84, 'The_Great_Gatsby'), (0.84, 'Shutter_Island'), (0.84, 'Parasite'), (0.84, 'Manchester_by_the_Sea'), (0.84, 'Her'), (0.83, 'Wolf_of_Wall_Street'), (0.82, 'Ready_Player_One'), (0.82, 'Midnight_in_Paris'), (0.79, 'Ex_Machina'), (0.79, '12_Years_a_Slave'), (0.78, 'La_La_Land'), (0.78, 'Black_Swan'), (0.77, 'Gone_Girl'), (0.76, 'Room'), (0.76, 'Now_You_See_Me'), (0.75, 'Toy_Story_3'), (0.74, 'Us'), (0.72, 'Once_Upon_a_Time_in_Hollywood'), (0.72, 'Dunkirk'), (0.71, 'The_Revenant'), (0.7, 'X-Men:_First_Class'), (0.7, 'The_Girl_with_the_Dragon_Tattoo'), (0.7, 'The_Farewell'), (0.66, 'The_Help'), (0.65, 'Three_Billboards_Outside_Ebbing'), (0.65, 'The_Hunger_Games'), (0.65, 'Spiderman:_Far_From_Home'), (0.65, 'Mad_Max:_Fury_Road'), (0.64, 'Hidden_Figures'), (0.63, 'Drive'),

In [17]:
import numpy as np

# Reading the movie titles and student IDs from text files.
movie_list = open('hw8_movies.txt').read().splitlines()
ids_list = open('hw8_ids.txt').read().splitlines()
# Loading the ratings from the file as strings.
rating_data = np.loadtxt('hw8_ratings.txt', dtype='str')

# Initializing a numerical array for ratings based on the string data.
numerical_ratings = np.zeros(rating_data.shape)

# Converting string ratings to numerical values (-1 for unwatched, 1 for liked, 0 for disliked).
for row in range(numerical_ratings.shape[0]):
    for col in range(numerical_ratings.shape[1]):
        if rating_data[row][col] == '?':
            numerical_ratings[row][col] = -1  # Indicates unwatched.
        else:
            numerical_ratings[row][col] = float(rating_data[row][col])

# Transpose the matrix so that movies are rows and students are columns.
numerical_ratings = numerical_ratings.T
# Boolean arrays for liked, disliked, and unwatched movies.
likes = numerical_ratings == 1
dislikes = numerical_ratings == 0
not_seen = numerical_ratings == -1

# Part a: Computing and displaying the mean popularity score for each movie.
mean_popularity = np.sum(likes, axis=1) / np.sum(likes + dislikes, axis=1)

# Creating a sorted list of movies by their mean popularity score.
ranked_popularity = [(np.round(score, 3), title) for score, title in zip(mean_popularity, movie_list)]
ranked_popularity.sort()

# Displaying the sorted list of movies.
sorted_movies = ", ".join([mv[1] for mv in ranked_popularity])
print("Sorted Movies by Popularity:\n", sorted_movies)

Sorted Movies by Popularity:
 Chappaquidick, I_Feel_Pretty, Fast_&_Furious:_Hobbs_&_Shaw, Magic_Mike, Bridemaids, The_Last_Airbender, World_War_Z, Fifty_Shades_of_Grey, Star_Wars:_The_Force_Awakens, The_Hateful_Eight, Terminator:_Dark_Fate, Drive, Pitch_Perfect, Phantom_Thread, Pokemon_Detective_Pikachu, Prometheus, Fast_Five, Good_Boys, The_Farewell, American_Hustle, The_Girl_with_the_Dragon_Tattoo, Avengers:_Age_of_Ultron, The_Shape_of_Water, The_Revenant, Thor, The_Help, Man_of_Steel, The_Perks_of_Being_a_Wallflower, Hidden_Figures, Frozen, Jurassic_World, Three_Billboards_Outside_Ebbing, Mad_Max:_Fury_Road, Captain_America:_The_First_Avenger, Dunkirk, Manchester_by_the_Sea, Ex_Machina, Rocketman, Ready_Player_One, Once_Upon_a_Time_in_Hollywood, 21_Jump_Street, Black_Swan, Iron_Man_2, Hustlers, Darkest_Hour, The_Hunger_Games, X-Men:_First_Class, Gone_Girl, Us, Avengers:_Endgame, La_La_Land, Avengers:_Infinity_War, Spiderman:_Far_From_Home, Midnight_in_Paris, Now_You_See_Me, 12_Years

In [23]:
# Part e: Expectation-Maximization Algorithm.

# Loading initial probabilities for ratings and movie-goer types.
initial_prob_R = np.loadtxt('hw8_probR_init.txt')
initial_prob_Z = np.loadtxt('hw8_probZ_init.txt')
initial_prob_Z = initial_prob_Z.reshape(-1,1)

print("Iteration  Likelihood:")
# Running the EM algorithm over 257 iterations.
for iteration in range(257):
    # Initializing arrays to hold probabilities for each type and movie.
    type_like_prob = np.zeros((4, 76, 258))
    type_dislike_prob = np.zeros((4, 76, 258))

    # Arrays to hold the first and second parts of the M-step calculations.
    part1_M_step = np.zeros((76,4))
    part2_M_step = np.zeros((76,4))

    # E-step: Calculating expected values for each movie-goer type.
    for z_type in range(4):
        type_like_prob[z_type] = np.multiply(initial_prob_R[:,z_type].reshape(-1,1), likes)
        type_dislike_prob[z_type] = np.multiply(1 - initial_prob_R[:,z_type].reshape(-1,1), dislikes)

    # Calculating the products where ratings exist.
    prob_product_likes = np.prod(type_like_prob, axis=1, where=(type_like_prob > 0))
    prob_product_dislikes = np.prod(type_dislike_prob, axis=1, where=(type_dislike_prob > 0))

    # Computing the numerator and denominator for the posterior probabilities.
    combined_numerator = np.multiply(np.multiply(prob_product_likes, prob_product_dislikes), initial_prob_Z)
    combined_denominator = np.sum(combined_numerator, axis=0).reshape(1,-1)
    posterior_probs = np.divide(combined_numerator, combined_denominator)

    # Logging the likelihood for certain iterations.
    if iteration in [0,1,2,4,8,16,32,64,128,256]:
        print(iteration,"        ",np.around(np.sum(np.log(combined_denominator))/258, decimals=2))

    # M-step: Updating probabilities based on current estimates.
    for z_type in range(4):
        part1_M_step[:, z_type] = np.sum(np.multiply(likes, posterior_probs[z_type].reshape(1, -1)), axis=1)
        temp_unseen = np.sum(np.multiply(not_seen, posterior_probs[z_type].reshape(1, -1)), axis=1)
        part2_M_step[:, z_type] = np.multiply(temp_unseen.reshape(-1, 1), initial_prob_R[:, z_type].reshape(-1, 1)).flatten()

    # Updating the probabilities for the movie rating given the movie-goer type.
    updated_prob_R = np.divide(part1_M_step + part2_M_step, np.sum(posterior_probs, axis=1).reshape(1,-1))
    updated_prob_Z = np.sum(posterior_probs, axis=1).reshape(-1,1)/258

    # Copy the updated probabilities back to the original variables.
    initial_prob_R = np.copy(updated_prob_R)
    initial_prob_Z = np.copy(updated_prob_Z)

Iteration  Likelihood:
0          -27.62
1          -18.48
2          -16.79
4          -15.55
8          -14.98
16          -14.68
32          -14.56
64          -14.53
128          -14.53
256          -14.52


In [30]:
# Part f: Generating personal movie recommendations based on the model.

# Multiplying the probabilities to get the scores for each movie.
hugo = student_ids.index('A59025239')
movie_scores = np.matmul(initial_prob_R, posterior_probs[:, hugo].reshape(-1,1))

# Creating a list of recommended movies by their scores, excluding already watched movies.
personal_recommendations = []
for idx, (title, score) in enumerate(zip(movie_list, movie_scores[:,0])):
    if numerical_ratings[idx][hugo] == -1:
        personal_recommendations.append((np.round(score, 2), title))

# Sorting the recommendations in descending order of scores.
personal_recommendations.sort(reverse=True)
print("Personal Recommendations:\n", personal_recommendations)

Personal Recommendations:
 [(0.93, 'The_Social_Network'), (0.9, 'Hustlers'), (0.87, 'Darkest_Hour'), (0.85, 'Les_Miserables'), (0.84, 'Shutter_Island'), (0.84, 'Parasite'), (0.84, 'Manchester_by_the_Sea'), (0.84, 'Her'), (0.79, 'La_La_Land'), (0.79, 'Ex_Machina'), (0.77, 'Now_You_See_Me'), (0.74, 'Us'), (0.71, 'The_Revenant'), (0.69, 'Frozen'), (0.66, 'The_Help'), (0.64, 'Drive'), (0.62, 'The_Hateful_Eight'), (0.58, 'The_Perks_of_Being_a_Wallflower'), (0.57, 'Rocketman'), (0.55, 'Phantom_Thread'), (0.54, 'Prometheus'), (0.53, 'Jurassic_World'), (0.5, 'Thor'), (0.5, 'I_Feel_Pretty'), (0.47, 'American_Hustle'), (0.44, 'Bridemaids'), (0.37, 'World_War_Z'), (0.37, 'Good_Boys'), (0.0, 'Chappaquidick')]


In [34]:
numerical_ratings[1][hugo]

-1.0